In [18]:
import pandas as pd
import random 
import os
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer

import os
import re

from datasets import load_dataset
import pandas as pd

import requests
import json

from statistics import mean
import numpy as np

import random
import csv
import nltk
from nltk.corpus import gutenberg

import random
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [53]:
def text_rank(text, compression_rate):
    sentences = sent_tokenize(text)
    sort_dict = {}
    for i in range(len(sentences)):
        sort_dict[i] = sentences[i]

    stop_words = set(stopwords.words('english'))
    preprocessed_sentences = []
    for sentence in sentences:
        words = word_tokenize(sentence.lower())
        words = [word for word in words if word.isalnum() and word not in stop_words]
        preprocessed_sentences.append(' '.join(words))
    
    vectorizer = CountVectorizer().fit_transform(preprocessed_sentences)
    similarity_matrix = cosine_similarity(vectorizer)
    
    damping_factor = 0.85  
    scores = np.ones(len(sentences))
    prev_scores = np.zeros(len(sentences))
    while np.sum(np.abs(scores - prev_scores)) > 0.001:
        prev_scores = np.copy(scores)
        for i in range(len(sentences)):
            scores[i] = (1 - damping_factor) + damping_factor * np.sum(similarity_matrix[:, i] * scores) / np.sum(similarity_matrix[:, i])
    ranked_sentences = sorted(([scores[i], sentence] for i, sentence in enumerate(sentences)), reverse=True)
    max_words = len(word_tokenize(text)) * (1-compression_rate)
    words=0
    chosen_texts = []
    for text in ranked_sentences:
        if words<=max_words:
            sen_length = len(word_tokenize(text[1])) # https://www.guru99.com/tokenize-words-sentences-nltk.html
            words += sen_length
            chosen_texts.append(text[1])
    chosen_dict = {}
    for i in range(len(sort_dict)):
        for text in chosen_texts:
            if sort_dict[i] == text:
                chosen_dict[i] = text
    summary_sentences = chosen_dict.values()
    summary = ' '.join(summary_sentences)
    return summary


In [54]:
print(text_rank('Das ist ein Test. Es ist warm. Das ist so. Es ist wild', 0.8))

[1.0, 'Es ist wild']
[1.0, 'Es ist warm.']
[1.0, 'Das ist so.']
[1.0, 'Das ist ein Test.']
['Es ist wild', 'Es ist warm.']
{1: 'Es ist warm.', 3: 'Es ist wild'}
Es ist warm. Es ist wild
Es ist warm. Es ist wild


In [3]:
df = pd.read_csv("../data/Result/reference_summaries_dataset.csv")
df.head()

,Text,Class,Summary
0,I have heard nothing from the Ambassador about...,Political speech,I have heard nothing from the Ambassador about...
1,I think it is in the public interest to procee...,Political speech,I think it is in the public interest to procee...
2,The A-11 aircraft now at Edwards Air force Bas...,Political speech,"for example, one of the most important technic..."
3,It is one of the most comprehensive bills in t...,Political speech,It is one of the most comprehensive bills in t...
4,"So long as there remains a man without a job, ...",Political speech,No American conscience can be at peace while a...


# Evaluierung

In [10]:
df1 = pd.read_csv("../data/Blogs_result/blogsresult_just_textrank.csv")
df2 = pd.read_csv("../data/Blogs_result/blogsresult_refsummarys.csv")


In [18]:
from rouge import Rouge

rouge = Rouge()

rouge_1_scores = []
rouge_2_scores = []
rouge_l_scores = []

for idx, row1 in df1.iterrows():
    summary1 = str(row1['Summary'])
    
    row2 = df2.loc[idx]
    summary2 = str(row2['Summary'])
    
    scores = rouge.get_scores(summary1, summary2)[0]
    
    rouge_1 = scores['rouge-1']['f']
    rouge_2 = scores['rouge-2']['f']
    rouge_l = scores['rouge-l']['f']
    
    rouge_1_scores.append(rouge_1)
    rouge_2_scores.append(rouge_2)
    rouge_l_scores.append(rouge_l)

min_rouge_1 = min(rouge_1_scores)
mean_rouge_1 = sum(rouge_1_scores) / len(rouge_1_scores)
max_rouge_1 = max(rouge_1_scores)

min_rouge_2 = min(rouge_2_scores)
mean_rouge_2 = sum(rouge_2_scores) / len(rouge_2_scores)
max_rouge_2 = max(rouge_2_scores)

min_rouge_l = min(rouge_l_scores)
mean_rouge_l = sum(rouge_l_scores) / len(rouge_l_scores)
max_rouge_l = max(rouge_l_scores)

print("Rouge-1 scores:")
print(f"Min: {min_rouge_1}")
print(f"Mean: {mean_rouge_1}")
print(f"Max: {max_rouge_1}")

print("Rouge-2 scores:")
print(f"Min: {min_rouge_2}")
print(f"Mean: {mean_rouge_2}")
print(f"Max: {max_rouge_2}")

print("Rouge-L scores:")
print(f"Min: {min_rouge_l}")
print(f"Mean: {mean_rouge_l}")
print(f"Max: {max_rouge_l}")

Rouge-1 scores:
Min: 0.0
Mean: 0.6212928899612717
Max: 0.999999995
Rouge-2 scores:
Min: 0.0
Mean: 0.5479318105729233
Max: 0.999999995
Rouge-L scores:
Min: 0.0
Mean: 0.615398270271077
Max: 0.999999995
